## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kiama,AU,-34.6833,150.8667,60.58,78,0,5.01,clear sky
1,1,Albany,US,42.6001,-73.9662,95.59,34,35,8.93,moderate rain
2,2,Ushuaia,AR,-54.8000,-68.3000,40.66,70,75,1.14,broken clouds
3,3,Castro,BR,-24.7911,-50.0119,68.52,50,81,5.70,broken clouds
4,4,Along,IN,28.1667,94.7667,75.16,98,100,0.92,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What should the minimum temperature be?"))
max_temp = float(input("What should the maximum temperature be?"))

What should the minimum temperature be?50
What should the maximum temperature be?70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

input_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
input_city_df.count()

City_ID                250
City                   250
Country                242
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Current Description    250
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
input_city_clean_df = input_city_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = input_city_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kiama,AU,60.58,clear sky,-34.6833,150.8667,
3,Castro,BR,68.52,broken clouds,-24.7911,-50.0119,
6,Cascais,PT,69.94,broken clouds,38.6979,-9.4215,
9,Thompson,CA,66.36,broken clouds,55.7435,-97.8558,
11,Port Alfred,ZA,59.63,scattered clouds,-33.5906,26.8910,
14,Busselton,AU,51.76,overcast clouds,-33.6500,115.3333,
16,Vardo,NO,51.78,broken clouds,70.3705,31.1107,
22,Marawi,PH,69.31,overcast clouds,7.9986,124.2928,
27,Mataura,NZ,57.47,overcast clouds,-46.1927,168.8643,
30,Cidreira,BR,68.38,scattered clouds,-30.1811,-50.2056,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [10]:
hotel_df.count()

City                   242
Country                242
Max Temp               242
Current Description    242
Lat                    242
Lng                    242
Hotel Name             242
dtype: int64

In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kiama,AU,60.58,clear sky,-34.6833,150.8667,The Sebel Kiama Harbourside
3,Castro,BR,68.52,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
6,Cascais,PT,69.94,broken clouds,38.6979,-9.4215,Hotel Vila Galé Cascais
9,Thompson,CA,66.36,broken clouds,55.7435,-97.8558,Burntwood Hotel
11,Port Alfred,ZA,59.63,scattered clouds,-33.5906,26.8910,The Halyards Hotel


In [12]:
# 8a. Create the output File (CSV)
# output_data_file = r'Module_6/Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))